1. Pobierz dane wejściowe.

In [9]:
import time
import pandas as pd
import requests
from datetime import datetime, timedelta

# Oblicz pierwszy i ostatni dzień zeszłego miesiąca
today = datetime.today()
first_day_of_current_month = datetime(today.year, today.month, 1)
last_day_of_last_month = first_day_of_current_month - timedelta(days=1)
first_day_of_last_month = datetime(last_day_of_last_month.year, last_day_of_last_month.month, 1)

# Konwertuj na UNIX timestamp
first_day = int(first_day_of_last_month.timestamp())
last_day = int(first_day_of_current_month.timestamp())

# Jeśli dzisiejszy dzień miesiąca jest większy niż 7, pobierz dane z bieżącego miesiąca
if today.day >= 7:
    first_day = int(first_day_of_current_month.timestamp())
    last_day = int((today + timedelta(days=1)).timestamp())

# Definicja nagłówków i danych do zapytania API
headers = {
    'X-BLToken': 'YOUR_TOKEN',
}
data = {
        'method': 'getInvoices',
}

# Pobierz wszystkie faktury
all_invoices = []
request = 0

while True:
    request += 1
    data['parameters'] = f'{{"date_from": {first_day}}}'
    response = requests.post('https://api.baselinker.com/connector.php', headers=headers, data=data)
    status = response.json().get('status')
    invoices = response.json().get('invoices', [])
    print(f'Request {request} status: {status}. Number of invoices: {len(invoices)}')
    
    if status == 'ERROR':
        print('Error! ')
        break

    if not invoices :
        print('No invoices?')
        break

    all_invoices.extend(invoices)
    first_day = invoices[-1]['date_add']
    if len(invoices) < 100 or first_day >= last_day:
        print('End of invoices.')
        break

    time.sleep(1)

# Zapisz dane do DataFrame
df = pd.DataFrame(all_invoices)


# Usuń niepotrzebne kolumny wg czasu
if 'date_add' in df.columns:
    df = df[df['date_add'] < last_day]

# Usuń duplikaty
df.drop_duplicates(subset=['invoice_id'], inplace=True)

Request 1 status: SUCCESS. Number of invoices: 100
Error! 


""


2. Sprawdź, czy dane są poprawne.

In [10]:
print('Zakres od dnia: ' + str(first_day_of_last_month))
print('Zakres do dnia: ' + str(first_day_of_current_month))
print('Liczba faktur: ' + str(len(df)))

df.info()

Zakres od dnia: 2024-08-01 00:00:00
Zakres do dnia: 2024-09-01 00:00:00
Liczba faktur: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 0 entries
Empty DataFrame


3. Zapisz dane do pliku XLSX

In [29]:
df.to_excel('invoices.xlsx', index=False)